In [1]:
import warnings
warnings.filterwarnings("ignore")

# About The data set
the data set is from the amazon product reviem of  books 

# Best Practices
1. Preprocessing
2. train andCleaning
3. bow,TFIDF,Word2vec
4. Train with Ml alogrithms

In [2]:
#Load the data set
import pandas as pd
df=pd.read_csv(r"C:\dl_projects\Kndel_reviem_sentiment_analysis\data\all_kindle_review.csv")
df.head()

,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


take the required data 

In [3]:
data=df[["reviewText","rating"]]
data.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4


In [4]:
data.shape

(12000, 2)

In [5]:
df.isnull().sum()

Unnamed: 0.1       0
Unnamed: 0         0
asin               0
helpful            0
rating             0
reviewText         0
reviewTime         0
reviewerID         0
reviewerName      38
summary            2
unixReviewTime     0
dtype: int64

In [6]:
data["rating"].unique()

array([3, 5, 4, 2, 1])

In [7]:
data["rating"].value_counts()

rating
5    3000
4    3000
3    2000
2    2000
1    2000
Name: count, dtype: int64

# Preprocessing  and Cleaning the text

separate the ratings into 2 value(encoding)

In [8]:
# postve reviem is one and negative reviem is 0
data["rating"]=data["rating"].apply(lambda x:0 if x<3 else 1)
data.head()


,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",1
1,Great short read. I didn't want to put it dow...,1
2,I'll start by saying this is the first of four...,1
3,Aggie is Angela Lansbury who carries pocketboo...,1
4,I did not expect this type of book to be in li...,1


In [9]:
data["rating"].unique()

array([1, 0])

In [10]:
data["rating"].value_counts()

rating
1    8000
0    4000
Name: count, dtype: int64

lower the all cases

In [11]:
data["reviewText"]=data["reviewText"].str.lower()

# removing the special characters and other special charcters

In [12]:
import re
from nltk.corpus import stopwords
from bs4 import BeautifulSoup


In [13]:
#removing special charcters
data["reviewText"]=data["reviewText"].apply(lambda x:re.sub('[^a-z A-Z 0-9]+',"",x))
#remove the url
data["reviewText"] = data["reviewText"].apply(lambda x: re.sub(r'(http|https|ftp|ssh)://[^\s]+', '', x))
# remove the html tags
data["reviewText"]=data["reviewText"].apply(lambda x:BeautifulSoup(x,"lxml").get_text())
#remove the any addtional sepaces
data["reviewText"]=data["reviewText"].apply(lambda x:" ".join(x.split()))




In [14]:
data.head()

,reviewText,rating
0,jace rankin may be short but hes nothing to me...,1
1,great short read i didnt want to put it down s...,1
2,ill start by saying this is the first of four ...,1
3,aggie is angela lansbury who carries pocketboo...,1
4,i did not expect this type of book to be in li...,1


lemmtizer

In [15]:
from nltk.stem import WordNetLemmatizer
lemmtizer=WordNetLemmatizer()

In [16]:
def lemmatize_words(text):
    return " ".join([lemmtizer.lemmatize(word=word) for word in text.split()])

In [17]:

data["reviewText"]=data['reviewText'].apply(lambda x: lemmatize_words(x))

In [18]:
data.head()

,reviewText,rating
0,jace rankin may be short but he nothing to mes...,1
1,great short read i didnt want to put it down s...,1
2,ill start by saying this is the first of four ...,1
3,aggie is angela lansbury who carry pocketbook ...,1
4,i did not expect this type of book to be in li...,1


split the train and test data

In [19]:
X=data["reviewText"]
Y=data["rating"]
X.shape

(12000,)

In [20]:
Y.shape

(12000,)

In [21]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=42)

# text embedding

Bag of Words

In [32]:
from sklearn.feature_extraction.text import CountVectorizer
bow=CountVectorizer()
x_train_bow_=bow.fit_transform(x_train).toarray()
x_test_bow_=bow.transform(x_test).toarray()

TF_IDF

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer()
x_train_tf=tfidf.fit_transform(x_train).toarray()
x_test_tf=tfidf.transform(x_test).toarray()

In [34]:
x_train_tf.shape

(9600, 37448)

In [35]:
x_train_bow_.shape

(9600, 37448)

In [36]:
x_test_bow_.shape

(2400, 37448)

# build the model

In [38]:
from sklearn.naive_bayes import GaussianNB
nb_model=GaussianNB()
model_bow=GaussianNB().fit(x_train_bow_,y_train)
model_tfidf=GaussianNB().fit(x_train_tf,y_train)

In [39]:

y_pred_bow=model_bow.predict(x_test_bow_)
y_pred_tfidf=model_tfidf.predict(x_test_tf)

In [45]:
from sklearn.metrics import classification_report,accuracy_score
accuracy_tfidf=accuracy_score(y_test,y_pred_tfidf)
accuracy_tfidf

0.5791666666666667

In [41]:
accuracy_bow=accuracy_score(y_test,y_pred_bow)
accuracy_bow

0.5779166666666666

In [47]:
print(classification_report(y_test,y_pred_tfidf))

              precision    recall  f1-score   support

           0       0.41      0.60      0.49       803
           1       0.74      0.57      0.64      1597

    accuracy                           0.58      2400
   macro avg       0.57      0.58      0.57      2400
weighted avg       0.63      0.58      0.59      2400

